In [32]:
import pandas as pd

# Ler o arquivo CSV
csv_file = r'C:\Users\mjzag\Documents\OBSCOOP\20231030_artigos_scopuswos_unicos.xlsx - 20231030_artigos_scopuswos_unicos.xlsx.csv'
df = pd.read_csv(csv_file)

# Nome do arquivo de saída BibTeX
bibtex_file = 'output.bib'

# Função para criar uma entrada BibTeX para cada linha no DataFrame
def create_bibtex_entry(row):
    authors = row['authors']
    if isinstance(authors, str):
        authors = authors.split('; ')
        author_list = ' and '.join(authors)
    else:
        author_list = 'Unknown'
    title = row['title']
    source_title = row['source_title']
    issn = row['issn']
    doi = row['doi']
    year = row['year']

    bibtex_entry = f"@article{{\n\
    author = {{{author_list}}},\n\
    title = {{{title}}},\n\
    journal = {{{source_title}}},\n\
    issn = {{{issn}}},\n\
    doi = {{{doi}}},\n\
    year = {{{year}}}\n\
}}\n"
    
    return bibtex_entry

# Aplicar a função de criação de entradas BibTeX a cada linha
df['bibtex_entry'] = df.apply(create_bibtex_entry, axis=1)

# Salvar as entradas BibTeX em um arquivo
with open(bibtex_file, 'w', encoding='utf-8') as f:
    for entry in df['bibtex_entry']:
        f.write(entry)
        f.write('\n')

print(f'Arquivo BibTeX gerado com sucesso: {bibtex_file}')


Arquivo BibTeX gerado com sucesso: output.bib
